In [3]:
import sys

import folium
import pandas as pd
import requests
from datetime import datetime

from api_key import api_key

In [4]:
url = "https://api.um.warszawa.pl/api/action/busestrams_get"

In [5]:
params = {
    "resource_id": "f2e5503e-927d-4ad3-9500-4ab9e55deb59",  # resource id given by the API
    "apikey": api_key,
    "type": 1,  # 1 - bus, 2 - tram
}

In [6]:
response = requests.get(url, params=params)

In [7]:
print(response.json())

{'result': [{'Lines': '161', 'Lon': 21.208984, 'VehicleNumber': '1000', 'Time': '2025-01-09 09:35:00', 'Lat': 52.16053, 'Brigade': '1'}, {'Lines': '196', 'Lon': 21.176939, 'VehicleNumber': '1001', 'Time': '2025-01-09 09:35:13', 'Lat': 52.256785, 'Brigade': '1'}, {'Lines': '219', 'Lon': 21.1588466, 'VehicleNumber': '1002', 'Time': '2025-01-09 09:35:14', 'Lat': 52.2123533, 'Brigade': '3'}, {'Lines': '196', 'Lon': 21.1771448, 'VehicleNumber': '1003', 'Time': '2025-01-09 09:35:14', 'Lat': 52.2568618, 'Brigade': '2'}, {'Lines': '219', 'Lon': 21.102243, 'VehicleNumber': '1004', 'Time': '2025-01-09 09:35:12', 'Lat': 52.222786, 'Brigade': '1'}, {'Lines': '305', 'Lon': 21.1615, 'VehicleNumber': '1005', 'Time': '2025-01-09 09:35:04', 'Lat': 52.225558, 'Brigade': '1'}, {'Lines': '311', 'Lon': 21.073384, 'VehicleNumber': '1006', 'Time': '2025-01-09 09:35:02', 'Lat': 52.254506, 'Brigade': '501'}, {'Lines': '219', 'Lon': 21.158661, 'VehicleNumber': '1007', 'Time': '2025-01-09 09:35:11', 'Lat': 52.18

In [8]:
print(len(response.json()["result"]))

1899


##### all are records recent?

In [113]:
how_long_ago = []

In [114]:
for bus in response.json()["result"]:
    lon = bus["Lon"]
    lat = bus["Lat"]
    lines = bus["Lines"]
    time = bus["Time"]
    number = bus["VehicleNumber"]
    brigade = bus["Brigade"]

    # parse time format: 2024-12-25 16:52:05
    time = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    diff = time.now() - time
    how_long_ago.append(diff)

In [115]:
print(how_long_ago)

[datetime.timedelta(seconds=81570, microseconds=855004), datetime.timedelta(days=2, seconds=26220, microseconds=855033), datetime.timedelta(seconds=46034, microseconds=855041), datetime.timedelta(days=1, seconds=57166, microseconds=855049), datetime.timedelta(days=2, seconds=60345, microseconds=855057), datetime.timedelta(seconds=45085, microseconds=855065), datetime.timedelta(days=2, seconds=63674, microseconds=855072), datetime.timedelta(seconds=21, microseconds=855079), datetime.timedelta(days=1, seconds=81195, microseconds=855087), datetime.timedelta(days=2, seconds=67625, microseconds=855094), datetime.timedelta(days=24, seconds=22989, microseconds=855102), datetime.timedelta(days=26, seconds=9558, microseconds=855109), datetime.timedelta(days=2, seconds=69504, microseconds=855117), datetime.timedelta(seconds=18, microseconds=855124), datetime.timedelta(days=41, seconds=4781, microseconds=855131), datetime.timedelta(seconds=21, microseconds=855138), datetime.timedelta(seconds=23, 

In [116]:
quantiles = [10, 25, 30, 35, 40, 50, 75, 90, 99]
for q in quantiles:
    print(f"{q}th percentile: {sorted(how_long_ago)[int(q/100*len(how_long_ago))]}")

10th percentile: 0:00:13.862814
25th percentile: 0:00:16.875243
30th percentile: 0:00:20.877481
35th percentile: 0:00:32.864842
40th percentile: 0:47:09.872325
50th percentile: 7:39:44.874629
75th percentile: 1 day, 23:50:43.871237
90th percentile: 3 days, 21:09:09.862618
99th percentile: 146 days, 23:43:10.877025


In [117]:
print(min(how_long_ago))
print(max(how_long_ago))

0:00:10.872113
287 days, 1:21:10.876457


##### size of response 

In [118]:
sys.getsizeof(str(response.json()["result"]))

240795

That is around 0.25MB per request

Daily (with request every 10 seconds) 6 * 60 * 24 * 0.25 = 2160MB 

approximately **2GB of raw data per day**

##### how frequently are records updated?

In [127]:
initial_response = requests.get(url, params=params).json()
initial_time = datetime.now()
for i in range(30):
    response = requests.get(url, params=params).json()
    if response != initial_response:
        print(f"Change detected after {datetime.now() - initial_time}")
        initial_response = response
        initial_time = datetime.now()

Change detected after 0:00:02.356654
Change detected after 0:00:12.393114
Change detected after 0:00:10.823231
Change detected after 0:00:12.437699
Change detected after 0:00:10.514227


### Saturday morning

In [278]:
url = "https://api.um.warszawa.pl/api/action/busestrams_get"
params = {
    "resource_id": "f2e5503e-927d-4ad3-9500-4ab9e55deb59",  # resource id given by the API
    "apikey": api_key,
    "type": 1,  # 1 - bus, 2 - tram
}
response = requests.get(url, params=params)
print(len(response.json()["result"]))

1894


the same number of records as on 2nd day of christmas

In [130]:
how_long_ago = []

for bus in response.json()["result"]:
    lon = bus["Lon"]
    lat = bus["Lat"]
    lines = bus["Lines"]
    time = bus["Time"]
    number = bus["VehicleNumber"]
    brigade = bus["Brigade"]

    # parse time format: 2024-12-25 16:52:05
    time = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    diff = time.now() - time
    how_long_ago.append(diff)

quantiles = [10, 25, 30, 35, 40, 45, 50, 75, 90, 99]
for q in quantiles:
    print(f"{q}th percentile: {sorted(how_long_ago)[int(q/100*len(how_long_ago))]}")

10th percentile: 0:01:43.624296
25th percentile: 0:01:46.621660
30th percentile: 0:01:46.635564
35th percentile: 0:01:49.626840
40th percentile: 0:01:56.628038
45th percentile: 0:02:33.631827
50th percentile: 1:48:07.627430
75th percentile: 22:53:28.624481
90th percentile: 4 days, 17:42:46.628435
99th percentile: 148 days, 19:22:16.642844


There is more recent buses than on 2nd day of christmas 45% vs 35%

#### Map of data in the city

In [150]:
# Create a map centered around Warsaw
warsaw_map = folium.Map(location=[52.2297, 21.0122], zoom_start=12)

# Add markers for each bus location
for bus in response.json()["result"]:
    if bus["Lines"] == "159":
        marker_color = (
            "red"
            if (
                datetime.now() - datetime.strptime(bus["Time"], "%Y-%m-%d %H:%M:%S")
            ).total_seconds()
            > 600
            else "blue"
        )
        folium.Marker(
            location=[bus["Lat"], bus["Lon"]],
            popup=f"Bus {bus['VehicleNumber']} on line {bus['Lines']}",
            icon=folium.Icon(color=marker_color, icon="circle"),
        ).add_to(warsaw_map)
        print(bus)

# Display the map
warsaw_map

{'Lines': '159', 'Lon': 20.882897667, 'VehicleNumber': '9517', 'Time': '2024-12-27 21:47:50', 'Lat': 52.2069405, 'Brigade': '4'}
{'Lines': '159', 'Lon': 20.883660667, 'VehicleNumber': '9521', 'Time': '2024-12-28 01:48:19', 'Lat': 52.206035833, 'Brigade': '3'}
{'Lines': '159', 'Lon': 20.9763385, 'VehicleNumber': '9522', 'Time': '2024-12-28 11:42:20', 'Lat': 52.221364667, 'Brigade': '1'}
{'Lines': '159', 'Lon': 21.047940833, 'VehicleNumber': '9534', 'Time': '2024-12-28 11:42:14', 'Lat': 52.209893167, 'Brigade': '2'}
{'Lines': '159', 'Lon': 21.067926167, 'VehicleNumber': '9538', 'Time': '2024-12-28 11:42:19', 'Lat': 52.2034145, 'Brigade': '3'}
{'Lines': '159', 'Lon': 20.989835, 'VehicleNumber': '9542', 'Time': '2024-12-28 11:42:20', 'Lat': 52.223743, 'Brigade': '4'}
{'Lines': '159', 'Lon': 20.954685, 'VehicleNumber': '9546', 'Time': '2024-12-28 11:42:24', 'Lat': 52.211905167, 'Brigade': '5'}
{'Lines': '159', 'Lon': 20.883432167, 'VehicleNumber': '9557', 'Time': '2024-12-06 10:03:57', 'Lat

I checked it with external sources and it seems that the data is up to date

there are 5 buses that are cruising around the city and they are in the right place \
the 3 remaining buses are in bus base

#### add data about bus stops

In [4]:
url = "https://api.um.warszawa.pl/api/action/dbstore_get"
params = {
    "id": "ab75c33d-3a26-4342-b36a-6e5fef0a3ac3",  # resource id given by the API
    "apikey": api_key,
}
stops = requests.get(url, params=params)
print(len(stops.json()["result"]))

8208


In [10]:
url = "https://api.um.warszawa.pl//api/action/public_transport_routes"
params = {
    "apikey": api_key,
}
routes = requests.get(url, params=params)
print(len(routes.json()["result"]))

327


In [11]:
routes.json()["result"]["159"].keys()

dict_keys(['TX-MSI', 'TX-BER', 'TX-BLC', 'TX-MSI-B', 'TP-BLC', 'TP-MSI'])

MSI - małe siekierki \
BER - berestecka \
BLC - blue city

In [12]:
routes.json()["result"]["159"]["TX-MSI"]["18"]

{'odleglosc': 7551,
 'ulica_id': '0326',
 'nr_zespolu': '7074',
 'typ': '1',
 'nr_przystanku': '01'}

In [13]:
for i in stops.json()["result"]:
    if i["values"][0]["value"] == "7074":
        print(i)

{'values': [{'value': '7074', 'key': 'zespol'}, {'value': '01', 'key': 'slupek'}, {'value': 'Zaruskiego', 'key': 'nazwa_zespolu'}, {'value': '0326', 'key': 'id_ulicy'}, {'value': '52.218719', 'key': 'szer_geo'}, {'value': '21.045546', 'key': 'dlug_geo'}, {'value': 'Szwoleżerów', 'key': 'kierunek'}, {'value': '2024-09-22 00:00:00.0', 'key': 'obowiazuje_od'}]}
{'values': [{'value': '7074', 'key': 'zespol'}, {'value': '02', 'key': 'slupek'}, {'value': 'Zaruskiego', 'key': 'nazwa_zespolu'}, {'value': '0326', 'key': 'id_ulicy'}, {'value': '52.218766', 'key': 'szer_geo'}, {'value': '21.045902', 'key': 'dlug_geo'}, {'value': 'Łazienkowska', 'key': 'kierunek'}, {'value': '2024-09-22 00:00:00.0', 'key': 'obowiazuje_od'}]}


In [28]:
rows = [
    {entry["key"]: entry["value"] for entry in item["values"]}
    for item in stops.json()["result"]
]
df = pd.DataFrame(rows)

df[df["zespol"] == "7074"]

,zespol,slupek,nazwa_zespolu,id_ulicy,szer_geo,dlug_geo,kierunek,obowiazuje_od
8033,7074,01,Zaruskiego,0326,52.218719,21.045546,Szwoleżerów,2024-09-22 00:00:00.0
8034,7074,02,Zaruskiego,0326,52.218766,21.045902,Łazienkowska,2024-09-22 00:00:00.0


In [13]:
routes.json()["result"].keys()

dict_keys(['L51', '217', 'N42', '212', '213', '210', '211', 'N45', 'N56', '264', 'N95', '218', '219', '133', '132', '131', '136', '135', '134', '139', '138', 'L14', '225', 'L52', '24', '25', '250', '27', '20', '22', '23', '735', '28', 'L39', '340', '221', '401', 'L23', 'N21', '220', '349', 'N25', '817', 'N85', 'N86', 'N81', 'N62', '719', '717', '716', '715', '714', '713', '712', '711', '710', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', 'L19', 'S4', 'L20', 'N71', 'N41', '736', 'N72', 'L25', 'N01', '379', 'L36', 'S40', '411', 'N31', 'N32', 'N02', 'N34', 'L12', '702', 'N37', '201', '319', '199', '198', '200', '704', '194', '197', '196', '191', '190', '193', '192', '115', '114', '117', '116', '111', '110', '112', 'N46', '119', '118', '204', '414', 'L18', '251', '207', 'L37', '256', 'L40', 'L34', '226', '525', '3', '527', '520', '521', '7', '523', 'L32', 'L28', 'E-2', 'N03', '723', '308', 'L50', 'L21', 'L31', '303', '26', '305', '245', '108', '240', '102', '103', '

Plot the route 159 on the map with stops and where the buses are

In [20]:
# Create a map centered around Warsaw
warsaw_map = folium.Map(location=[52.2297, 21.0122], zoom_start=12)

# Add markers for each bus location
for bus in response.json()["result"]:
    if bus["Lines"] == "159":
        if (
            datetime.now() - datetime.strptime(bus["Time"], "%Y-%m-%d %H:%M:%S")
        ).total_seconds() < 600:
            folium.Marker(
                location=[bus["Lat"], bus["Lon"]],
                popup=f"Bus {bus['VehicleNumber']} on line {bus['Lines']}",
                icon=folium.Icon(color="blue", icon="circle"),
            ).add_to(warsaw_map)

# Add markers for each bus stop on route 159
for stop in routes.json()["result"]["159"]["TX-MSI"].values():
    przystanek = df[
        (df["zespol"] == stop["nr_zespolu"]) & (df["slupek"] == stop["nr_przystanku"])
    ]
    folium.Circle(
        location=[przystanek["szer_geo"].values[0], przystanek["dlug_geo"].values[0]],
        popup=przystanek["nazwa_zespolu"].values[0]
        + " "
        + przystanek["slupek"].values[0],
        radius=10,
        color="green",
        fill=True,
    ).add_to(warsaw_map)

for stop in routes.json()["result"]["159"]["TP-MSI"].values():
    przystanek = df[
        (df["zespol"] == stop["nr_zespolu"]) & (df["slupek"] == stop["nr_przystanku"])
    ]
    folium.Circle(
        location=[str(float(przystanek["szer_geo"].values[0]) + 0.0001), przystanek["dlug_geo"].values[0]],
        popup=przystanek["nazwa_zespolu"].values[0]
        + " "
        + przystanek["slupek"].values[0],
        radius=10,
        color="red",
        fill=True,
    ).add_to(warsaw_map)
# Display the map
warsaw_map

### Summary

Some endpoints are in polish some in english

The data is up to date and is mostly valid - I have checked it with external sources and forums relates to public transport in Warsaw

Data about location is updated every 10 seconds

Other endpoints are updated every day

What matters for us is:
- vehicle_location
- get_stops
- public_transport_route
- get_line_timetable 

base on that I can estimate delay of the bus

bellow I will attach the code used to get the data

### Stops

In [283]:
url = "https://api.um.warszawa.pl/api/action/dbstore_get"
params = {
    "id": "ab75c33d-3a26-4342-b36a-6e5fef0a3ac3",  # resource id given by the API
    "apikey": api_key,
}
stops = requests.get(url, params=params)
print(len(stops.json()["result"]))

8241


### Routes

In [284]:
url = "https://api.um.warszawa.pl//api/action/public_transport_routes"
params = {
    "apikey": api_key,
}
routes = requests.get(url, params=params)
print(len(routes.json()["result"]))

287


### Location

In [285]:
url = "https://api.um.warszawa.pl/api/action/busestrams_get"
params = {
    "resource_id": "f2e5503e-927d-4ad3-9500-4ab9e55deb59",  # resource id given by the API
    "apikey": api_key,
    "type": 1,  # 1 - bus, 2 - tram
}
locations = requests.get(url, params=params)
print(len(locations.json()["result"]))

1896


### Timetable

In [291]:
url = "https://api.um.warszawa.pl//api/action/dbtimetable_get"
params = {
    "id": "e923fa0e-d96c-43f9-ae6e-60518c9f3238",  # resource id given by the API
    "apikey": api_key,
    "busstopId": "7074",
    "line": "159",
    "busstopNr": "02",
}
timetable = requests.get(url, params=params)
print(len(timetable.json()["result"]))

51
